<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/preProcessamento_noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
from sklearn.model_selection import train_test_split
import io
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

### Upload da base de noticias

In [2]:
#csvNoticias = files.upload()  
!wget https://raw.githubusercontent.com/HedersonSantos/Noticias/main/noticia_tratada.zip
!unzip noticia_tratada.zip
%ls



--2021-05-30 14:27:00--  https://raw.githubusercontent.com/HedersonSantos/Noticias/main/noticia_tratada.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6432391 (6.1M) [application/zip]
Saving to: ‘noticia_tratada.zip’

noticia_tratada.zip 100%[===================>]   6.13M  18.0MB/s    in 0.3s    

2021-05-30 14:27:01 (18.0 MB/s) - ‘noticia_tratada.zip’ saved [6432391/6432391]

Archive:  noticia_tratada.zip
  inflating: noticia_tratada.csv     
noticia_tratada.csv  noticia_tratada.zip  sample_data/


In [3]:
#dfNews_all = pd.read_csv(io.BytesIO(csvNoticias['noticia_tratada.csv']), header=0, delimiter=';')
dfNews_all = pd.read_csv('noticia_tratada.csv', header=0, delimiter=';')
print(dfNews_all.columns)
dfNews_all.shape

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO'],
      dtype='object')


(3994, 9)

### Tratamento da CATEGORIA

In [4]:
print('Dimensao Inicial:',dfNews_all.shape)
#Elimina noticias duplicadas
dfNews = dfNews_all.drop_duplicates(subset=['URL'],keep='first').copy()
#Elimina noticias sem texto
dfNews = dfNews[(~dfNews['TEXT_TRATADO'].isnull())]
print('Dimensão após eliminar duplicadas e textos em branco:',dfNews.shape)
#Tira as marcações do campo Text[] do Postgres
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: x['CATEGORIA'].replace('[','').replace(']','').replace('"','').replace(", 'redacao'",'').replace("'",''),axis=1)
#Agrupa as notícias locais como categoria LOCAL
siglaEstados = ['ac','al','ap','am','ba','ce','df','es','go','ma','mt','ms','mg','pa','pb','pr','pe','pi','rj','rn','rs','ro','rr','sc','sp','se','to']
nomeEstados = ['acre','alagoas','amapa','amazonas','bahia','ceara','distrito-federal','espirito-santo','goias','maranhao','mato-grosso','mato-grosso-sul','minas-gerais','minas',
               'para','paraiba','parana','pernambuco','piaui','rio-janeiro','rio-grande-norte','rio-grande-sul','rondonia','roraima','santa-catarina','sao-paulo','sergipe','tocantins']
nomeCidades =['rio','bh',]
local = siglaEstados + nomeEstados + nomeCidades
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'local' if x['CATEGORIA'] in local else x['CATEGORIA'],axis=1)
#efe, afp, rfi são notícias internacionais. 
mundo = ['efe','afp','rfi','mundo','internacional']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'mundo' if x['CATEGORIA'] in mundo else x['CATEGORIA'],axis=1)
#Na lista miscelanea há as categorias que possuem miscelâneas de noticias de outras categorias.
miscelanea = ['deutschewelle', '','bbc','ansa','universa']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'miscelanea' if x['CATEGORIA'] in miscelanea else x['CATEGORIA'],axis=1)
#'reuters' é uma categoria especifica do UOL. Possui textos de economia e miscelânea de noticias.
dfreuters = dfNews[dfNews['CATEGORIA']=='reuters'].copy()
dfreuters['CATEGORIA'] = dfreuters.loc[:,['CATEGORIA','URL']].apply(lambda x: 'economia' if 'economia' in x['URL']  else 'miscelanea',axis=1)
dfNews=dfNews[dfNews['CATEGORIA']!='reuters']
dfNews = pd.concat([dfNews,dfreuters],sort=False)
#noticias é uma categoria da globo. Traz noticias do Jornal Nacional. São miscelâneas.
dfnoticia = dfNews[dfNews['CATEGORIA']=='noticia'].copy()
dfnoticia['CATEGORIA'] = dfnoticia.loc[:,['CATEGORIA','URL']].apply(lambda x: 'saude' if 'ciencia-e-saude' in x['URL']  else 'miscelanea',axis=1)
dfNews=dfNews[dfNews['CATEGORIA']!='noticia']
dfNews = pd.concat([dfNews,dfnoticia],sort=False)
#redacao é uma categoria da uol. Traz noticias do meio-ambiente. VERIFICAR QUANDO ATUALIZAR O DATASET SE PERMANECE ASSIM.
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA','URL']].apply(lambda x: 'meio-ambiente' if ('meio-ambiente' in x['URL']) and
                                                              (x['CATEGORIA']=='redacao') else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA','URL']].apply(lambda x: 'meio-ambiente' if ('meio-ambiente' in x['URL']) and
                                                              (x['CATEGORIA']=='sociedade') else 
                                                              'miscelanea' if x['CATEGORIA']=='sociedade' else x['CATEGORIA'] ,axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'saude' if x['CATEGORIA'] in ['vivabem','bemestar']  else x['CATEGORIA'] ,axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'esporte' if x['CATEGORIA'] in ['futebol','volei'] else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'midia' if x['CATEGORIA'] in ['famosos','cinema','oscar','videos','bbb','musica','lives'], else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'economia' if x['CATEGORIA'] in ['economia'], else x['CATEGORIA'],axis=1)
dfNews.shape

Dimensao Inicial: (3994, 9)
Dimensão após duplicadas e textos em branco: (3991, 9)


(3991, 9)

In [ ]:
#Filtra categorias com poucas noticias. 
dfNewsGrupo = dfNews.groupby(['CATEGORIA'])['URL'].count().reset_index().sort_values('URL',ascending=False)
dfNewsGrupo = dfNewsGrupo[dfNewsGrupo['URL']<=10]
dfNews = dfNews[~dfNews['CATEGORIA'].isin(dfNewsGrupo['CATEGORIA'].tolist())]
dfNews.shape

(2884, 9)

In [5]:
dfNewsGrupo = dfNews.groupby(['CATEGORIA'])['URL'].count().reset_index().sort_values('URL',ascending=False)
dfNewsGrupo

,CATEGORIA,URL
30,local,673
47,politica,583
36,mundo,522
34,miscelanea,350
54,saude,287
23,esporte,251
17,economia,219
5,brasil,189
14,coronavirus,172
33,midia,161


### Prepara DataSet Treinamento, Teste

In [ ]:
dfMiscelanea = dfNews[dfNews['CATEGORIA']=='miscelanea'].loc[:,['TEXT_TRATADO']]
dfMiscelanea.shape

(279, 1)

In [ ]:
dfNews.columns

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO'],
      dtype='object')

In [ ]:
#vamos obter uma amostra randômica de 130 noticias para cada CATEGORIA e dividir em dois datasets Treinamento e Teste.
n=130
categorias = dfNewsGrupo[dfNewsGrupo['URL']>n].loc[:,['CATEGORIA']]['CATEGORIA'].to_list()
frame=[]
for categoria in categorias:
  dftmp = dfNews[dfNews['CATEGORIA']==categoria].loc[:,['TEXTO','CATEGORIA']].sample(n=n)
  frame.append(dftmp)
dfAmostra = pd.concat(frame,ignore_index=True, sort=False)
print('Amostra:', dfAmostra.shape)

df_train, df_test = train_test_split(dfAmostra, train_size = 0.9, random_state = 10)
print('Treinamento:',df_train.shape)
print('Teste:', df_test.shape)


Amostra: (1430, 2)
Treinamento: (1287, 2)
Teste: (143, 2)
